In [1]:
import sys
import torch
sys.path.append('/home/zengdun/FedLab/')
import torchvision
import torchvision.transforms as transforms
from copy import deepcopy
import random
import os 

os.environ["CUDA_VISIBLE_DEVICES"] = "1"


In [2]:
# unit test of `SerialHandler`
from fedlab_utils.dataset.sampler import DistributedSampler
from fedlab_core.client.serial_handler import SerialHandler
from models.lenet import LeNet
from fedlab_utils.aggregator import Aggregators
from fedlab_utils.serialization import SerializationTool
from fedlab_utils.functional import evaluate

model = LeNet().cuda()
aggregator = Aggregators.fedavg_aggregate
criterion = torch.nn.CrossEntropyLoss()

trainset = torchvision.datasets.MNIST(
    root='/home/zengdun/datasets/mnist/', train=True, download=True, transform=transforms.ToTensor())

testset = torchvision.datasets.MNIST(
    root='/home/zengdun/datasets/mnist/', train=False, download=True, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(testset, batch_size=len(testset), drop_last=False, shuffle=False)

sim_client_num = 100  # client总数

test_handler = SerialHandler(local_model=deepcopy(model), aggregator=aggregator, dataset=trainset, sim_client_num=sim_client_num)

model_params = SerializationTool.serialize_model(model)

to_select = [i+1 for i in range(sim_client_num)] # client_id 从1开始

for global_epoch in range(2):
    selection = random.sample(to_select, 10)
    print(selection)
    model_params = test_handler.train(epochs=10, batch_size=128, idx_list=selection, model_parameters=model_params, cuda=True)
    SerializationTool.deserialize_model(model, model_params)
    evaluate(model, criterion, test_loader, cuda=True)


INFO:root:starting training process of client [51]
INFO:root:Client[51] Traning. Epoch 1/10, Loss 11.5145, Time 0.16s
[51, 52, 20, 44, 79, 24, 7, 53, 90, 64]
INFO:root:Client[51] Traning. Epoch 2/10, Loss 11.5005, Time 0.10s
INFO:root:Client[51] Traning. Epoch 3/10, Loss 11.4867, Time 0.12s
INFO:root:Client[51] Traning. Epoch 4/10, Loss 11.4730, Time 0.12s
INFO:root:Client[51] Traning. Epoch 5/10, Loss 11.4593, Time 0.10s
INFO:root:Client[51] Traning. Epoch 6/10, Loss 11.4456, Time 0.11s
INFO:root:Client[51] Traning. Epoch 7/10, Loss 11.4316, Time 0.09s
INFO:root:Client[51] Traning. Epoch 8/10, Loss 11.4174, Time 0.09s
INFO:root:Client[51] Traning. Epoch 9/10, Loss 11.4029, Time 0.09s
INFO:root:Client[51] Traning. Epoch 10/10, Loss 11.3880, Time 0.09s
INFO:root:starting training process of client [52]
INFO:root:Client[52] Traning. Epoch 1/10, Loss 11.5210, Time 0.09s
INFO:root:Client[52] Traning. Epoch 2/10, Loss 11.5018, Time 0.09s
INFO:root:Client[52] Traning. Epoch 3/10, Loss 11.483